In [2]:
#Importing libraries
from os import environ, chdir

import keras
from keras.preprocessing.image import ImageDataGenerator

from keras import models, layers, optimizers, callbacks
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization,AveragePooling2D
from keras.models import Sequential
import matplotlib.pyplot as plt
import numpy
from numpy import set_printoptions

from keras.utils import to_categorical
from keras import models, layers
chdir (r'E:\Capstone\split-20200229T043656Z-001')
set_printoptions(precision=4,suppress=True)

Using TensorFlow backend.


In [3]:
# CNN Architecture
cnn4 = Sequential()
cnn4.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(64, 64,3)))
cnn4.add(MaxPooling2D(pool_size=(2, 2)))

cnn4.add(Conv2D(32, kernel_size=(3, 3),strides=(1, 1), activation='relu'))
cnn4.add(MaxPooling2D(pool_size=(2, 2)))


cnn4.add(Flatten())

cnn4.add(Dense(output_dim=128, activation='relu'))
cnn4.add(Dense(output_dim=1, activation='sigmoid'))
print(cnn4.summary())


# Model Loss function and Optimizer method
compile = cnn4.compile (loss=keras.losses.binary_crossentropy,
                            optimizer=keras.optimizers.Adam(),
                            metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6272)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               802944    
________________________________________________

C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
  if sys.path[0] == '':
C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1)`
  del sys.path[0]


In [3]:
# Setting Image and Data Generators
train_idg = ImageDataGenerator (rescale=1. / 255,
                                horizontal_flip=True,
                                zoom_range=0.2,
                                shear_range=0.2,
                                width_shift_range=0.1,
                                height_shift_range=0.1)

train_g = train_idg.flow_from_directory (directory=r'E:\Capstone\split-20200229T043656Z-001\split\train',
                                         target_size=(64,64),class_mode='binary',
                                         batch_size=32)

valid_idg = ImageDataGenerator (rescale=1. / 255)

valid_g = valid_idg.flow_from_directory (directory=r'E:\Capstone\split-20200229T043656Z-001\split\validation',
                                         target_size=(64,64),class_mode='binary',
                                         batch_size=32)

Found 2654 images belonging to 2 classes.
Found 883 images belonging to 2 classes.


In [4]:
# Training Options
fit = cnn4.fit_generator (generator=train_g, steps_per_epoch=20, epochs=50, verbose=1,
                              validation_data=valid_g, validation_steps=4)

Instructions for updating:
Use tf.cast instead.
Epoch 1/50
20/20 [==============================] - 19s 953ms/step - loss: 0.6115 - accuracy: 0.7188 - val_loss: 0.3666 - val_accuracy: 0.8906
Epoch 2/50
20/20 [==============================] - 19s 964ms/step - loss: 0.3076 - accuracy: 0.8859 - val_loss: 1.3245 - val_accuracy: 0.6562
Epoch 3/50
20/20 [==============================] - 13s 646ms/step - loss: 0.1398 - accuracy: 0.9641 - val_loss: 0.9066 - val_accuracy: 0.6719
Epoch 4/50
20/20 [==============================] - 12s 605ms/step - loss: 0.0440 - accuracy: 0.9953 - val_loss: 1.1784 - val_accuracy: 0.6328
Epoch 5/50
20/20 [==============================] - 6s 325ms/step - loss: 0.0337 - accuracy: 0.9953 - val_loss: 2.3978 - val_accuracy: 0.5547
Epoch 6/50
20/20 [==============================] - 5s 242ms/step - loss: 0.0434 - accuracy: 0.9859 - val_loss: 0.7633 - val_accuracy: 0.6797
Epoch 7/50
20/20 [==============================] - 5s 241ms/step - loss: 0.0534 - accuracy: 0.9

In [8]:
#Evaluating model
eval_idg = ImageDataGenerator (rescale=1. / 255)
eval_g = eval_idg.flow_from_directory (directory=r'E:\Capstone\split-20200229T043656Z-001\split\test',
                                         target_size=(64,64),class_mode='binary',
                                         batch_size=32,shuffle=False)
eval_loss,eval_acc=cnn4.evaluate_generator(generator=eval_g,steps=1)
print('Evaluation loss over never seen images is : {:.4f}' .format(eval_loss))
print('Evaluation accuracy over never seen images is : {:4.2f}%' .format(eval_acc*100))

Found 40 images belonging to 2 classes.
Evaluation loss over never seen images is : 0.1066
Evaluation accuracy over never seen images is : 93.75%


In [4]:
# Saving Model
cnn4.save (filepath=r'lead_zinc classification.h5', overwrite=True)